In [45]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize

In [46]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yasin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yasin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yasin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [47]:
df= pd.read_csv("C:/Users/yasin/Desktop/otel-restoran-tahmini/csv/stemmed_sentences.csv")

In [48]:
df

,fri terrif also hot crisp love food server maria filet mignon impecc mussel steak tartar also v seafood cocktail wonder larg lump crabmeat delici lobster penn al pomodoro bucatini cacio e pepe highli recommend onion soup mussel french great food good price best cocktail worth travel mile incred italian perfect pizza top splendid pizza excel french onion soup food great especi mussel good fresh
0,abso love charcuteri board burger cours onion ...
1,authent delici korean cuisin exampl sea urchin...
2,best steak vibe citi stand dish main lobster s...
3,coq au vin tasti moist chicken serv delic sauc...
4,pasta delici toni chick french toast waffl scr...
...,...
8678,overal experi good
8679,everyth good
8680,ac room servic good
8681,like thank manag


In [49]:
df.columns = ["0"]

# NaN değerleri ve boş stringleri temizle
df = df.dropna()
df = df[df["0"].str.strip() != ""]

# Temizlenmiş ilk 5 cümleyi göster
print(df.head())

                                                   0
0  abso love charcuteri board burger cours onion ...
1  authent delici korean cuisin exampl sea urchin...
2  best steak vibe citi stand dish main lobster s...
3  coq au vin tasti moist chicken serv delic sauc...
4  pasta delici toni chick french toast waffl scr...


In [50]:
#Varsayalım ki DataFrame'iniz df ve yorumlar '0' sütununda
text = ' '.join(df['0'].astype(str).tolist())  # Tüm yorumları tek bir metin haline getir
sentences = sent_tokenize(text)  # Metni cümlelere ayır

In [51]:
# Stopwords listesini almak
stop_words = set(stopwords.words('english'))

In [55]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize

# Lemmatizer ve Stemmer nesnelerini oluştur
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
# Kelimeleri tokenleştirip, lemmatize etme ve stemleme
def preprocess_sentence(sentence):
    tokens = word_tokenize(sentence)
    filtered_tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    return lemmatized_tokens, stemmed_tokens

# Kullanım örneği
tokenized_corpus_lemmatized = []
tokenized_corpus_stemmed = []

for sentence in sentences:
    lemmatized_tokens, stemmed_tokens = preprocess_sentence(sentence)
    tokenized_corpus_lemmatized.append(lemmatized_tokens)
    tokenized_corpus_stemmed.append(stemmed_tokens)

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [57]:
# Ön işlenmiş token listelerini tekrar metne çeviriyoruz
lemmatized_texts = [' '.join(tokens) for tokens in tokenized_corpus_lemmatized]

lemmatized_texts[:3]


['abso love charcuteri board burger cours onion soup chicken tikka masala favorit good ramen frill wing crispi duck fri rice far favorit delici breakfast sandwich cou buratta arancini bucatini lasagna everyth fish fresh moist lamb highli recommend branzino ravioli adult devour cevich octopu enjoy g everi bite authent delici korean cuisin exampl sea urchin bibimbab black cod clam chicken satay bun start follow prawn pad thai fantast beef excel food bolognes follow pork chop soup slightli salti pork noodl insid great qualiti price start load superb steak realli delici chicken piccata casual dinner best brunch fluffiest french toast ribey steak excel qualiti meat beauti cook ribey steak excel qualiti meat beauti cook given complimentari pistachio tiramisu light fluffi crun plea duck pizza kale salad rich chocol p taco salad nyc pizzeria amaz best steak vibe citi stand dish main lobster salad main best risotto ever live thu love small cozi delici great authent american breakfast kitchen fa

In [58]:
# TF-IDF vektörizerı başlatıyoruz
vectorizer = TfidfVectorizer()

# TF-IDF matrisini oluşturuyoruz
#terim frekansları, belge frekanslarıni hesplar
#TF-IDF vektörlerine dönüştürür
tfidf_matrix = vectorizer.fit_transform(lemmatized_texts)

## Kelimeleri alalım
#F-IDF vektörleştirme işleminde kullanılan tüm kelimelerin essiz bir listesini döndürur
feature_names = vectorizer.get_feature_names_out()

# TF-IDF matrisini pandas DataFrame'e çevir-gorunurluk acisindan- calismasi kolay
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# İlk birkaç satırı gösterelim-ilk 5 cümle
print(tfidf_df.head())

#Her satır bir cümleyi temsil eder
#Her sütun bir kelimeyi temsil eder
#Hücreler ise o kelimenin o cümledeki TF-IDF skorudur - her cumle icin degisir-bakiniz:slaytlar

        aak    aakash      aall   aaminti       aap      aaps   aashram  \
0  0.000138  0.000277  0.000138  0.000138  0.000415  0.000138  0.000138   

     aayush      abac   abandon  ...     カジュアル        外人      早餐好吃  熱水量又大又舒服  \
0  0.000138  0.000138  0.000277  ...  0.000138  0.000138  0.000138  0.000138   

        第一次      감자튀김       맛있고       바삭한      샐러드도       편하고  
0  0.000138  0.000138  0.000138  0.000138  0.000138  0.000138  

[1 rows x 5629 columns]


In [ ]:
tfidf_df.to_csv('tfidf_stemmed.csv', index=False)